<a href="https://colab.research.google.com/github/arponbasu/ITSP_Noise_Reduction_Project_2ndSem/blob/master/DataFilesGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gTTS
!pip install pydub
print("Process Completed...")

Process Completed...


In the code below we generate both clean (by applying a text to Speech API on a sentence database) and noisy audio clips to be used as training and testing datasets for a neural network model designed for denoising the clips.

gTTS is the Google Text to Speech API in python, and pydub is an audio processing library. We'll use both quite extensively below.

In [ ]:
import IPython
from scipy.io import wavfile
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import librosa
%matplotlib inline
print("Process Completed...")

Process Completed...


Given below is a demonstration of the gTTS library (https://www.geeksforgeeks.org/convert-text-speech-python/). The tld argument stands for the accent/ dialect we want to hear in speech. co.in represents "Indian English". 

In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import os
language = 'en'

mytext = 'My name is Arpon Basu.'
myobj = gTTS(text=mytext,tld='co.in', lang=language, slow=False)
myobj.save("welcome.mp3")
sound = AudioSegment.from_mp3("welcome.mp3")
sound.export("welcome.wav", format="wav")
print("Process Completed...")

Process Completed...


Given below is the process used for rendering a wav file into an audio clip to be played within the notebook itself. This was taken from https://timsainburg.com/noise-reduction-python.html, also given in the references. 

In [ ]:
import IPython
wav_loc = "welcome.wav"
src_rate, src_data = wavfile.read(wav_loc)
IPython.display.Audio(data = src_data, rate = src_rate)

Below we have opened a file named 'ActualSent.txt' containing 10,000 sentences, which was taken from the Tatoeba database (https://tatoeba.org/en/downloads). 
Those contained indexed sentences along with a tag 'eng', which were spliced and removed.
Note that the file Actual sent.txt was uploaded through the library google.colab from our local devices. However, since that piece of code would stall the serial execution (as it would ask you to upload that file from your device, we haven't included that cell).
Anyways, since that file is there in the home directory, you can download it for your further perusal.

In [ ]:
file1 = open('ActualSent.txt', 'r')
Lines = file1.readlines()
 
count = 0
sents = []

for line in Lines:
    count += 1
    index = line.strip().find('eng') 
    str = (line.strip())[index:]
    str = str[4:]
    sents.append(str)
    print(str)

print("Process Completed...")

Streaming output truncated to the last 5000 lines.
The holiday continues to be very boring.
I'm dying to see Kumiko.
Kumi did not talk about her club.
Hearing this song after so long really brings back the old times.
After a long absence he came back.
An old friend of mine dropped in on me for the first time in ages.
It's been so long since we last met up. Let's have a drink or two and talk about the good old days.
I haven't seen him for a long time.
Please forgive me for not having written for a long time.
Will Mr Oka teach English?
After running up the hill, I was completely out of breath.
The hills are bathed in sunlight.
The hill lay covered with snow.
The hill slopes downward to the river.
The hill was all covered with snow.
The hill glows with autumnal colors.
The hill is always green.
You see a white building at the foot of the hill.
The wind blew harder yet when we reached the top of the hill.
The building which stands on the hillside is our school.
A castle stands a little way

The following function assigns an accent based on the (language) code passed to it. The language in all 4 cases however, remains English.

In [ ]:
def AssignAccent(code):
  if code == 1: return 'com.au' #Australian Accent
  elif code == 2: return 'co.uk' #British Accent
  elif code == 3: return 'com' #American Accent
  elif code == 4: return 'co.in' #Indian Accent
  else: return 'co.in' #Indian Accent
print("Process Completed...")

Process Completed...


Note that ideally we should've (would've?) parsed the entire file 'ActualSent.txt' to obtain our datasets. However, since the file is too large, we get a HTTP error 429 (from the gTTs API) when we pass the entire file.
Thus we've limited ourselves to just 500 here, but one can just change that ceiling here (to anything less than or equal to 10,000) and the rest of the code will work perfectly.

In [ ]:
UpperLimit = 500
print("Process Completed...")

Process Completed...


Here we create serialized file names through a list comprehension, and store the speech verion of each sentence into the corresponding file.
Note that 'cap' has really no use here, and one could as well replace it with (len(sents)+1), but it was pre-emptively put in place just to prevent crashes.
Also note that the accent of speech is chosen randomly.

In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import os
import random

count = 0
cap = int(0.125*len(sents)+1)
file_names_mp3=['file{}.mp3'.format(i) for i in range(1,cap)] #len(sents)+1
file_names_wav=['file{}.wav'.format(i) for i in range(1,cap)]#len(sents)+1

#print(file_names)
print("Saving sentences in  mp3 format....")
for sentence in sents:
   ind = sents.index(sentence)
   if ind > UpperLimit:
     break
   myobj = gTTS(text = sentence, tld = AssignAccent(random.randint(1, 4)), lang = 'en', slow = False)
   myobj.save(file_names_mp3[ind])
   
print("Saving sentences in  mp3 format complete")

Saving sentences in  mp3 format....
Saving sentences in  mp3 format complete


Note that most sound processing application files deal with wav files, and hence the mp3 files generated by gTTs have to be converted first.

In [ ]:
print("Converting  mp3 format to .wav format....")
iterator = 0
for audio_mp3 in file_names_mp3:
   iterator = iterator + 1
   if iterator > UpperLimit + 1:
     break
   sound = AudioSegment.from_mp3(audio_mp3)
   sound.export(file_names_wav[file_names_mp3.index(audio_mp3)], format="wav")
print("Convertion of all  mp3s to wav is complete")

Converting  mp3 format to .wav format....
Convertion of all  mp3s to wav is complete


A 100 different types of noise commonly encountered in real life (obtained from http://web.cse.ohio-state.edu/pnl/corpus/HuNonspeech/HuCorpus.html) were uploaded above as noise files. Since they weren't seen on the LHS file section, the following cell 
was run to confirm that the n1.wav, n2.wav,...,n100.wav noise files have indeed been uploaded.

In [ ]:
!ls -a

.		file245.mp3  file391.mp3  file85.mp3	     noisy_clip272.wav
..		file245.wav  file391.wav  file85.wav	     noisy_clip273.wav
ActualSent.txt	file246.mp3  file392.mp3  file86.mp3	     noisy_clip274.wav
.config		file246.wav  file392.wav  file86.wav	     noisy_clip275.wav
file100.mp3	file247.mp3  file393.mp3  file87.mp3	     noisy_clip276.wav
file100.wav	file247.wav  file393.wav  file87.wav	     noisy_clip277.wav
file101.mp3	file248.mp3  file394.mp3  file88.mp3	     noisy_clip278.wav
file101.wav	file248.wav  file394.wav  file88.wav	     noisy_clip279.wav
file102.mp3	file249.mp3  file395.mp3  file89.mp3	     noisy_clip27.wav
file102.wav	file249.wav  file395.wav  file89.wav	     noisy_clip280.wav
file103.mp3	file24.mp3   file396.mp3  file8.mp3	     noisy_clip281.wav
file103.wav	file24.wav   file396.wav  file8.wav	     noisy_clip282.wav
file104.mp3	file250.mp3  file397.mp3  file90.mp3	     noisy_clip283.wav
file104.wav	file250.wav  file397.wav  file90.wav	     noisy_clip284.wav
file105.mp3	

The following function assigns SNR based on the mode given to it. It's used below in generating noisy files with varying intensities of noise.

In [ ]:
def AssignSNR (mode,scale=1):
  if mode <= 4: return (scale*mode)
  else: return 20
print("Process Completed...")

Process Completed...


The following code, also inspired from https://timsainburg.com/noise-reduction-python.html, generates as many "noisy files" as clean ones, by randomly mixing with it a noise file (from n1.wav, n2.wav, ..., n100.wav) in a randomly chosen ratio (which is the SNR), and the resulting files are also saved.

In [ ]:
import random
from scipy.io.wavfile import write

corrupt_clip_wav = ['noisy_clip{}.wav'.format(k) for k in range(1,UpperLimit+2)]

for j in range(1, UpperLimit + 2):
    
    wav_loc = 'file{}.wav'.format(j)
    noise_loc = 'n{}.wav'.format(random.randint(1, 100))

    src_rate, src_data = wavfile.read(wav_loc)
    src_data = src_data / 32768
    
    noise_rate, noise_data = wavfile.read(noise_loc)
  
  # get some noise to add to the signal
    rem = (len(src_data))%(len(noise_data))
    quotient = len(src_data) // len(noise_data) 
    noise_to_add = noise_data[:rem]
    for i in range(quotient):
        noise_to_add = np.concatenate([noise_to_add, noise_data])
    noise_to_add = noise_to_add / max(noise_to_add)
  
  
  #  apply noise
    snr = AssignSNR(random.randint(1,4))  # signal to noise ratio
    corrupt_clip = src_data + noise_to_add / snr
    
    scaled = np.int16(corrupt_clip/np.max(np.abs(corrupt_clip)) * 32767)
    write(corrupt_clip_wav[j-1], src_rate, scaled)
    
    
print("Process Completed...")

Process Completed...


In [ ]:
import IPython
wav_loc = "noisy_clip231.wav" #A test example
src_rate, src_data = wavfile.read(wav_loc)
IPython.display.Audio(data = src_data, rate = src_rate)

75% of both the clean and noisy files are saved in the training folders Y and X respectively (the notation is consistent with the literature on CycleGANs, which is eventually the reason for which these files are being created), while the remaining 25% files are placed in the test folders.

In [ ]:
import shutil

StartTrain = 1
EndTrain = int(0.75*UpperLimit)
StartTest = int(0.75*UpperLimit) + 1
EndTest = UpperLimit + 1

for i in range(StartTrain,EndTrain + 1):
  shutil.copyfile('file{}.wav'.format(i),'./Y/file{}.wav'.format(i))
  shutil.copyfile('noisy_clip{}.wav'.format(i),'./X/file{}.wav'.format(i))
for i in range(StartTest,EndTest + 1):
  shutil.copyfile('file{}.wav'.format(i),'./test_Y/file{}.wav'.format(i))
  shutil.copyfile('noisy_clip{}.wav'.format(i),'./test_X/file{}.wav'.format(i))
print("Process Completed...")

Process Completed...


In [ ]:
!ls -a ./Y

.	     file168.wav  file236.wav  file304.wav  file373.wav
..	     file169.wav  file237.wav  file305.wav  file374.wav
file100.wav  file16.wav   file238.wav  file306.wav  file375.wav
file101.wav  file170.wav  file239.wav  file307.wav  file37.wav
file102.wav  file171.wav  file23.wav   file308.wav  file38.wav
file103.wav  file172.wav  file240.wav  file309.wav  file39.wav
file104.wav  file173.wav  file241.wav  file30.wav   file3.wav
file105.wav  file174.wav  file242.wav  file310.wav  file40.wav
file106.wav  file175.wav  file243.wav  file311.wav  file41.wav
file107.wav  file176.wav  file244.wav  file312.wav  file42.wav
file108.wav  file177.wav  file245.wav  file313.wav  file43.wav
file109.wav  file178.wav  file246.wav  file314.wav  file44.wav
file10.wav   file179.wav  file247.wav  file315.wav  file45.wav
file110.wav  file17.wav   file248.wav  file316.wav  file46.wav
file111.wav  file180.wav  file249.wav  file317.wav  file47.wav
file112.wav  file181.wav  file24.wav   file318.wav  file48.wav
f

References used for the above are given below, to the best of my knowledge:
1. Sound processing and Noise addition: https://timsainburg.com/noise-reduction-python.html
2. Noise files Database:
http://web.cse.ohio-state.edu/pnl/corpus/HuNonspeech/HuCorpus.html
3. Some additional noise files (not used here) can be found here: https://www.ecs.utdallas.edu/loizou/speech/noizeus/
4. Sentence Database : https://tatoeba.org/en/downloads
5. Basic Introduction to gTTs : https://www.geeksforgeeks.org/convert-text-speech-python/
6. A guide to using tld in gTTs : https://gtts.readthedocs.io/en/latest/module.html#examples
7. Why the UpperLimit is required : https://cloud.google.com/speech-to-text/quotas
8. Some Audio processing trivia : https://stackoverflow.com/questions/10357992/how-to-generate-audio-from-a-numpy-array

